## Create obsids to work on

In [ ]:
img_names = pd.read_table('/Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt',
                          header=None, squeeze=True)
img_names.head()

In [ ]:
from planet4 import region_data

In [ ]:
img_names = region_data.Ithaca.season2

img_names

In [ ]:
from ipyparallel import Client
c = Client()

lbview = c.load_balanced_view()
dview = c.direct_view()

In [ ]:
with c[:].sync_imports():
    from planet4.projections import hi2mos

In [ ]:
results = lbview.map_async(hi2mos, img_names)

In [ ]:
from nbtools import display_multi_progress

In [ ]:
display_multi_progress(results, img_names)

In [ ]:
for res in results:
    print(res)

## cleanup

In [ ]:
!rm -f *RED*.cub

# xy2latlon

In [ ]:
from pysis.isis import campt
from pysis.exceptions import ProcessError
import pvl

In [ ]:
from pathlib import Path
edrpath = Path('/Volumes/Data/planet4/season2_3_EDRs/')
clusterpath = Path('/Users/klay6683/Dropbox/data/planet4/inca_seasons2and3_v2/')

In [ ]:
obsids = !cat /Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt

In [ ]:
fpaths = [item for obsid in obsids for item in (clusterpath.glob("{}_*.csv".format(obsid)))]

In [ ]:
fpaths[0]

In [ ]:
blotch_coords = ['', 'p1', 'p2', 'p3', 'p4']
fan_coords = ['', 'arm1', 'arm2']

In [ ]:
from ipyparallel import Client
c = Client()

lbview = c.load_balanced_view()
dview = c.direct_view()

In [ ]:
with dview.sync_imports():
    from pysis.isis import campt
    from pysis.exceptions import ProcessError
    from pathlib import Path
    from ipyparallel import CompositeError

In [ ]:
def do_campt(mosaicname, savepath, temppath):
    try:
        campt(from_=mosaicname, to=savepath, format='flat', append='no',
              coordlist=temppath, coordtype='image')
    except ProcessError as e:
        print(e.stderr)
        return obsid, False


def process_inpath(inpath, marking, mosaicpath):
    coords_switch = dict(blotches=blotch_coords,
                         fans=fan_coords)
    
    df = pd.read_hdf(str(inpath), 'df')
    for coord in coords_switch[marking]:
        print("Coord", coord)
        if coord == '':
            name = 'base'
            tempcoords = ['x', 'y']
        else:
            name = coord
            tempcoords = [coord + '_x', coord + '_y']
        print("Tempcoords", tempcoords)
        temppath = inpath.with_suffix('.tocampt')
        try:
            df[tempcoords].to_csv(str(temppath), header=False, index=False)
        except KeyError:
            return False
        print("name", name)
        savename = "{stem}_{c}_campt_out.csv".format(stem=inpath.stem, c=name)
        print("savename", savename)
        savepath = clusterpath / savename
        try:
            do_campt(mosaicpath, savepath, temppath)
        except:
            return False
    return True

def obsid_marking_from_clusterpath(inpath):
    s = inpath.stem  # stem is 'name' (without folders) minus extension
    # s = ESP_xxxxxx_yyyy_blotches (or ..._fans)
    # obsid = s[:until last '_']
    sep = s.rfind('_')
    return s[:sep], s[sep+1:]

edrpath = Path('/Volumes/Data/planet4/season2_3_EDRs/')

def marking_mosaicpath_from_inpath(inpath):
    obsid, marking = obsid_marking_from_clusterpath(inpath)
    mosaicname = 'redMosaic' + obsid + '.cub'
    mosaicpath = edrpath / mosaicname
    return marking, mosaicpath
    
def xy2latlon(inpath):
    d = dict(inpath=inpath)
    ok = process_inpath(inpath, *marking_mosaicpath_from_inpath(inpath))
    d['ok'] = ok
    return d

In [ ]:
dview.push(dict(process_inpath=process_inpath,
                do_campt=do_campt,
                blotch_coords=blotch_coords,
                fan_coords=fan_coords,
                clusterpath=Path('/Users/klay6683/Dropbox/data/planet4/'
                                 'inca_s23_0.5cut_applied/')))

In [ ]:
results = lbview.map_async(xy2latlon, fpaths)

In [ ]:
from iuvs.multitools import nb_progress_display

nb_progress_display(results, fpaths)

In [ ]:
res = pd.DataFrame(results.result)

In [ ]:
res.ok.value_counts()

In [ ]:
res[res.ok==False].inpath.values

# Combining campt results

In [ ]:
p = fpaths[0]

In [ ]:
class GroundMarking(object):
    def __init__(self, resultfile):
        self.p = Path(resultfile)
        
        # this loop creates filename paths for all coords campt output files
        # and assigns them to object attributes, like
        # self.basefile, self.p1file, etc.
        self.paths = []
        self.mapped_coords = []
        for coord in self.coords:
            path = self.campt_fname(coord)
            setattr(self, coord+'file', path)
            self.paths.append(path)
            self.store_mapped_coords(coord, path)
        self.mapped_coords = pd.concat(self.mapped_coords, axis=1)
        newpath = self.p.with_name(self.p.stem+'_latlons.csv')
        self.mapped_coords.to_csv(str(newpath), index=False)
        self.coordspath = newpath

    def campt_fname(self, coordname):
        return self.p.with_name(self.p.stem + '_{}_campt_out.csv'.format(coordname))
    
    def store_mapped_coords(self, coord, path):
        df = pd.read_csv(path)
        subdf = df[['PlanetographicLatitude',
                    'PositiveEast360Longitude']]
        subdf.columns = [coord+'_lat', coord+'_lon']
        self.mapped_coords.append(subdf)

class GroundBlotch(GroundMarking):
    coords = ['base', 'p1', 'p2', 'p3', 'p4']
    kind = 'blotch'


class GroundFan(GroundMarking):
    coords = ['base', 'arm1', 'arm2']
    kind = 'fan'

    
def get_ground_marking(fname):
    tokens = Path(fname).stem.split('_')
    if tokens[-1] == 'blotches':
        return GroundBlotch(fname)
    else:
        return GroundFan(fname)

In [ ]:
for path in fpaths:
    print(path.stem)
    get_ground_marking(path)

# Corner coordinates

In [ ]:
from planet4 import io
from pysis.isis import campt
import pvl


class CornerCalculator(object):
    img_x_size = 840
    img_y_size = 648
    def __init__(self, cubepath):
        self.cubepath = cubepath
        self.get_tiles_max(self.img_name)
        
    def transform(self, x, y, xtile, ytile):
        x_offset = self.img_x_size - 100
        y_offset = self.img_y_size - 100
        x_HiRISE = x + ((x_offset)*(xtile-1))#**formula
        y_HiRISE = y + ((y_offset)*(ytile-1))#**formula
        return x_HiRISE, y_HiRISE

    @property
    def img_name(self):
        s = Path(self.cubepath).stem
        return s[9:]

    @property
    def UL(self):
        return (1, 1)
        
    @property
    def LL(self):
        return self.transform(1, self.img_y_size, 1, self.yt_max)
    
    @property
    def UR(self):
        return self.transform(self.img_x_size, 1, self.xt_max, 1)
    
    @property
    def LR(self):
        return self.transform(self.img_x_size, self.img_y_size,
                             self.xt_max, self.yt_max)
    
    def get_tiles_max(self, img_name):
        db = io.DBManager()
        data = db.get_image_name_markings(img_name)
        self.xt_max = data.x_tile.max()
        self.yt_max = data.y_tile.max()

    def get_lats_lon(self, s=None, l=None):
        try:
            gp = pvl.load(campt(from_=str(self.cubepath),
                                sample=s, 
                                line=l)).get('GroundPoint')
        except ProcessError as e:
            print(e.stdout)
            print(e.stderr)
            raise e
        lat_centric = gp['PlanetocentricLatitude']
        lat_graphic = gp['PlanetographicLatitude']
        lon = gp['PositiveWest360Longitude']
        return dict(lat_centric=lat_centric,
                    lat_graphic=lat_graphic,
                    lon=lon)
    
    def calc_corners(self):
        d = {}
        for corner in ['UL','LL','UR','LR']:
            try:
                coords = self.get_lats_lon(*getattr(self, corner))
            except ProcessError as e:
                print(self.img_name)
                print("Corner:", corner)
                print(e.stdout)
                print(e.stderr)
                continue
            for k, v in coords.items():
                d[corner+'_'+k] = v
        return pd.DataFrame(d, index=[self.img_name])

In [ ]:
edrpath

In [ ]:
cubepaths = !ls /Volumes/Data/planet4/season2_3_EDRs/*.cub

In [ ]:
cornercalc = CornerCalculator(cubepaths[0])

In [ ]:
cornercalc.calc_corners()

In [ ]:
bucket = []
for cube in cubepaths:
    print(Path(cube).stem)
    cc = CornerCalculator(cube)
    bucket.append(cc.calc_corners())

In [ ]:
p = '/Volumes/Data/planet4/season2_3_EDRs/redMosaicESP_011729_0985.cub'

In [ ]:
meta = pd.read_hdf('/Users/klay6683/Dropbox/SternchenAndMe/python_stuff/hirise_rdr_index.hdf')

In [ ]:
results = pd.concat(bucket)

In [ ]:
results.index.name = 'img_name'

In [ ]:
results.to_csv('inca_city_corners.csv')

In [ ]:
pd.read_csv('inca_city_corners.csv', index_col=0)